# Document Classification and Translation Notebook

This notebook enables classifying and translating document descriptions stored in an archive based on the RIC-O classification system and Netzer & Schuster's suggested vocabulary of document types. It leverages OpenAI's GPT-4 model to perform tasks such as translation, classification, and metadata extraction.

## Features

- **Interactive UI:** Select operations, add new classifications, and control processing modes.
- **Batch Processing with Checkpointing:** Processes data in batches and saves progress incrementally to prevent data loss due to runtime disconnections.
- **Detailed Feedback:** Provides summaries post-processing, including success rates and error logs.
- **Robust Data Handling:** Ensures consistent and accurate categorization of data.

## Prerequisites

Ensure you have the necessary API keys for OpenAI and Google Books. These should be securely stored and retrieved within the notebook.

In [ ]:
# Step 1: Install necessary libraries
%%capture
!pip install pandas openai==0.28 requests google-auth google-auth-oauthlib google-auth-httplib2 pydantic tqdm ipywidgets

## Import Libraries
Import all necessary libraries for data processing, API interactions, and UI components.

In [ ]:
# Step 2: Import necessary libraries
import pandas as pd
from google.colab import drive, userdata
import openai
import requests
import json
from collections import defaultdict
from pydantic import BaseModel, ValidationError
from tqdm import tqdm  # Progress bar
import time
import ipywidgets as widgets
from IPython.display import display, clear_output
import logging
import shutil

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')



## Mount Google Drive
Mount your Google Drive to access input and output files directly from the Colab environment.

In [ ]:
# Step 3: Mount Google Drive
drive.mount('/content/drive')

## Retrieve API Keys Securely
Fetch API keys for OpenAI from Colab's secret management to ensure security.

In [ ]:
# Step 4: Retrieve API keys securely from Colab's secret management
openai_api_key = userdata.get('openai_api_key')

# Initialize OpenAI client with API key
openai.api_key = openai_api_key

## Define Pydantic Model for Response Validation
This model ensures that the JSON response from the OpenAI API adheres to the expected structure.

In [ ]:
class TranslationClassificationModel(BaseModel):
    german_translation: str = ""
    english_translation: str = ""
    item_class: str = ""
    document_type: str = ""
    format: str = ""
    language: str = ""
    creation_place: str = ""
    creation_date: str = ""
    publication_date: str = ""
    number_of_pages: str = ""
    documentary_form_type: str = ""
    production_technique_type: str = ""
    main_subject: str = ""
    record_state: str = ""
    title: str = ""


## Define Classification Lists
These lists define the possible classes, document types, formats, languages, production techniques, and record states used for classification.

In [ ]:
# Step 6: Define possible classes, document types, formats, languages, and record states
classes = [
    "Certificate", "Communication Document", "Creative Fiction", "Creative Nonfiction", "EgoDocument", "Ephemera", "Image",
    "Legal Document", "Report", "Medical Document", "Financial_Document", "Organization Papertrail"
]
document_types = [
    "מכתב|Letter",
    "מעטפה|Envelope",
    "מסמך מתורגם|Translated Document",
    "רשימה|List",
    "קבלה|Receipt",
    "שיר|Poem",
    "כרטיס ברכה|Greeting Card",
    "פתק|Note",
    "גלויה|Postcard",
    "כרטיס ביקור|Business Card",
    "תצלום|Photograph",
    "פנקס|Notepad",
    "חוברת|Booklet",
    "תעודת רישום|Registry Certificate",
    "הזמנה|Invitation",
    "מברק|Telegram",
    "הרצאה|Lecture",
    "נאום|Speech",
    "חומר לימודי|School Material",
    "תעודה|Certificate",
    "תעודה מנהלית|Administrative Certificate",
    "עדות|Testimony",
    "דוח|Report",
    "תעודת לימודים|Academic Certificate",
    "קורות חיים|Biography",
    "מאמר מדעי|Academic Paper",
    "תוכניית כנס|Conferences Program",
    "מאמר|Article",
    "קריקטורה|Caricature",
    "ספר|Book",
    "עיתון|Newspaper",
    "גזיר עיתון|News Clipping",
    "טיוטה|Draft",
    "ייפוי כוח|Power of Attorney",
    "תעודת חבר|Membership Certificate",
    "כתובה|Ketubah",
    "הסכם|Agreement",
    "טופס|Form",
    "ניירות מכתבים|Stationery",
    "ברושור|Brochure",
    "מזכרת|Souvenir",
    "ציור|Painting",
    "לא מזוהה|Unrecognized",
    "טבלה|Table",
    "כתב עת|Serial",
    "אינפוגרפיקה|Infographics",
    "כרטיס|Ticket|Card",
    "בול|Stamp",
    "אלמנט גרפי|Graphic Element",
    "לוגו|Logo",
    "מפה|Map",
    "תג|Tag",
    "תלוש|Voucher",
    "תכניה|Playbill",
    "קטלוג|Catalog",
    "מדריך|Manual",
    "סימניה|Bookmark",
    "קומנטר|Commentary",
    "מרשם רפואי|Medical Prescription",
    "מתכון|Recipe",
    "סקיצה|Sketch",
    "משחק|Game",
    "בקשה|Application",
    "מודעת אבל|Death Notice",
    "דף אינטרנט|Web Page",
    "המחאה|Check",
    "חוזה|Contract",
    "הקדשה|Dedication",
    "דוא׳ל|Email",  # Corrected entry using geresh
    "צוואה|Will",
    "חפץ|Object",
    "יומן|Diary",
    "לוח שנה|Calendar",
    "סיפור|Fiction",
    "תווים|Scores",
    "תוכן דתי|Religious Material",
    "רשימות|Personal Notes",
    "סיכום פגישה|Minutes",
    "קולאז'|Collage",
    "חשבונית|Invoice",
    "מסמך מורכב|Composite Document",
    "הסכם|Agreement",
    "שומר מקום|Place holder",
    "שרבוט|Doodles",
    "אוגד|Binder",
    "תכנית אדריכלית|Floor Plan",
    "צרור|Bundle",
    "ספר זכרונות|Memory Book"
]

formats = [
    "Handwritten|כתב-יד", "Typewritten|הדפסה", "Print|דפוס", "Visual Element|אלמנט ויזואלי", "Unknown|לא ידוע"
]

languages = [
    "Hebrew|עברית", "English|אנגלית", "German|גרמנית", "Other|אחר", "Unspecified|לא צוין", "Irrelevant|לא רלוונטי"
]

production_technique_types = [
    "print", "handwriting", "drawing", "typewriting", "photography", "born-digital", "other"
]

record_states = [
    "draft", "original", "copy", "other"
]

In [ ]:

## Step 7: Load the Excel File and Extract Column Names for Dropdowns

# Define file paths (ensure these paths are correct)
input_file_path = '/content/drive/MyDrive/JeckeItems/parsed-items-cleaned.xlsx'  # Update with your file path

try:
    df = pd.read_excel(input_file_path)
    logging.info(f"Loaded input file successfully with columns: {df.columns.tolist()}")
except FileNotFoundError:
    logging.error(f"Input file not found at path: {input_file_path}")
    df = pd.DataFrame()  # Empty DataFrame to prevent errors

# Extract column names for dropdowns
column_names = df.columns.tolist()


## Step 8: Create Dropdowns for Selecting 'item_id' and 'item_description' Columns

# Dropdown for selecting 'item_id' column
item_id_dropdown = widgets.Dropdown(
    options=column_names,
    description='Item ID Column:',
    disabled=False,
)

# Dropdown for selecting 'item_description' column
item_description_dropdown = widgets.Dropdown(
    options=column_names,
    description='Item Description Column:',
    disabled=False,
)

# Display the dropdowns
display(widgets.HTML("<h2>Select Column Mappings:</h2>"))
display(item_id_dropdown)
display(item_description_dropdown)

## Prompt Building Function
Constructs a detailed prompt based on selected operations to send to the OpenAI API. It dynamically includes tasks such as translation, classification, and metadata extraction based on user selections.

In [ ]:
def build_prompt(description, operations, classes, document_types, formats, languages, production_technique_types, record_states):
    prompt = "Please perform the following tasks for the given Hebrew description of the item:\n\n"

    keys = []
    task_number = 1  # Keep track of task numbers in the prompt
    if operations['translate_german']:
        prompt += f"{task_number}. **Translate** the Hebrew description to **German**.\n"
        keys.append("german_translation")
        task_number += 1
    if operations['translate_english']:
        prompt += f"{task_number}. **Translate** the Hebrew description to **English**.\n"
        keys.append("english_translation")
        task_number += 1
    if operations['format']:
        prompt += f"{task_number}. **Determine the format** of the document. Choose from: {', '.join(formats)}.\n"
        prompt += "Note: When the description mentions 'הדפסה' in the context of letters or documents, it usually refers to 'Typewritten', not 'Print'. Do not assume 'Print' just because letters are involved. If unsure, choose 'Unknown' or 'Handwritten'.\n"
        keys.append("format")
        task_number += 1
    if operations['language']:
        prompt += f"{task_number}. **Identify the language(s)** of the document itself. Choose from: {', '.join(languages)}. If multiple languages apply, separate them with a pipe `|`. If the language is not specified or not applicable (e.g., for visual elements with no text), respond with 'Unspecified' or 'Irrelevant' as appropriate.\n"
        keys.append("language")
        task_number += 1
    if operations['creation_date']:
        prompt += f"{task_number}. **Determine the creation date** of the document. Provide the most precise date available, in one of the following formats: YYYY-MM-DD, YYYY-MM, or YYYY. Do not fill in missing parts with default values. If the date is not present or cannot be determined, respond with an empty string \"\".\n"
        keys.append("creation_date")
        task_number += 1
    if operations['publication_date']:
        prompt += f"{task_number}. **Determine the publication date** of the document. Provide the most precise date available, in one of the following formats: YYYY-MM-DD, YYYY-MM, or YYYY. Do not fill in missing parts with default values. If the date is not present or cannot be determined, respond with an empty string \"\".\n"
        keys.append("publication_date")
        task_number += 1
    if operations['number_of_pages']:
        prompt += f"{task_number}. **Determine the number of pages** the document has, if mentioned. If not mentioned, respond with an empty string \"\".\n"
        keys.append("number_of_pages")
        task_number += 1
    if operations['documentary_form_type']:
        prompt += f"{task_number}. **Documentary form type:** Indicate 'yes' if the document has a documentary form, otherwise leave it as an empty string \"\".\n"
        keys.append("documentary_form_type")
        task_number += 1
    if operations['production_technique_type']:
        prompt += f"{task_number}. **Specify the production technique type** of the document. Choose from: {', '.join(production_technique_types)}. If not applicable, leave it as an empty string \"\".\n"
        keys.append("production_technique_type")
        task_number += 1
    if operations['main_subject']:
        prompt += f"{task_number}. **Identify the main subject** of the document, if it can be deduced. Only if the main subject is a name of an indivdual or institution. If not, respond with an empty string \"\".\n"
        keys.append("main_subject")
        task_number += 1
    if operations['record_state']:
        prompt += f"{task_number}. **Record state:** Indicate the state of the document. Choose from: {', '.join(record_states)}.\n"
        prompt += "Note: For identifying a 'copy', look for Hebrew terms like 'צילום מכתב', 'תצלום מכתב', 'העתק', 'עותק'.\n"
        keys.append("record_state")
        task_number += 1
    if operations['title']:
        prompt += f"{task_number}. **Provide a title** for the archival item, containing only essential information based on the description.\n"
        keys.append("title")
        task_number += 1
    if operations['item_class']:
        prompt += f"{task_number}. **Suggest the most fitting class** for the item from this list: {', '.join(classes)}.\n"
        keys.append("item_class")
        task_number += 1
    if operations['document_type']:
        prompt += f"{task_number}. **Suggest the most fitting document type(s)** for the item from this list: {', '.join(document_types)}. If multiple types apply, separate them with a pipe `|`.\n"
        keys.append("document_type")
        task_number += 1

    # Append the actual description
    prompt += f"\n**Description:**\n{description}\n\n"

    prompt += "\n**Important:** Return the result as a structured JSON object with the following keys:\n"
    for key in keys:
        prompt += f"- \"{key}\"\n"

    prompt += "\n**Note:** All attributes pertain to the item described, not to the description itself.\n\n"
    prompt += "Example Output:\n"
    example = {key: "Example Value" for key in keys}
    prompt += f"{json.dumps(example, indent=4, ensure_ascii=False)}\n"

    # **Important Addition:** Instruct the model not to include code fences
    prompt += "\n**Important:** Do not include any markdown, code fences, or additional text. Return only the JSON object."
    prompt += "\n**Ensure that all keys are included in the JSON output, even if their values are empty strings.**\n"

    return prompt, keys

## OpenAI API Interaction Functions
These functions handle communication with the OpenAI API, including error handling and ensuring robust data parsing.

In [ ]:
def extract_and_translate(description, operations, classes, document_types, formats, languages, production_technique_types, record_states):
    prompt, keys = build_prompt(description, operations, classes, document_types, formats, languages, production_technique_types, record_states)

    # Optionally, print the prompt for debugging
    # print("Prompt being sent to OpenAI API:\n", prompt)

    response = openai.ChatCompletion.create(
        model="gpt-4o",  # Corrected model name
        messages=[{
            "role": "user",
            "content": prompt
        }],
        temperature=0.2,  # Lower temperature for more deterministic output
        max_tokens=1500  # Increased max tokens to accommodate more fields
    )

    # Extract the assistant's reply
    assistant_reply = response['choices'][0]['message']['content'].strip()

    return assistant_reply, keys

def extract_and_translate_skip_on_failure(description, operations, classes, document_types, formats, languages, production_technique_types, record_states):
    try:
        return extract_and_translate(description, operations, classes, document_types, formats, languages, production_technique_types, record_states)
    except openai.error.OpenAIError as e:
        logging.error(f"OpenAI API error for description: {description[:30]}... Error: {e}. Skipping...")
        return None, []
    except Exception as e:
        logging.error(f"Unexpected error for description: {description[:30]}... Error: {e}. Skipping...")
        return None, []

## JSON Parsing Function
Safely parses the JSON response from the OpenAI API, ensuring all expected keys are present and handling any malformed JSON by returning default empty values.

In [ ]:
def safe_eval(x, keys):
    if x is None:
        return {key: "" for key in keys}
    try:
        # Remove code fences if present
        if x.startswith("```"):
            x = "\n".join(x.split("\n")[1:-1])
        result = json.loads(x)
        # Validate against Pydantic model for robustness
        validated = TranslationClassificationModel(**result)
        # Convert validated data back to dict
        return validated.dict()
    except (json.JSONDecodeError, TypeError, ValidationError) as e:
        logging.error(f"JSON decoding/validation error: {e}")
        logging.error(f"Raw response: {x[:30]}...")  # Helps in debugging
        return {key: "" for key in keys}


## Functions to Add New Classes and Document Types
Allow users to dynamically add new classes and document types through the UI.

In [ ]:
def add_new_class(new_class):
    new_class = new_class.strip()
    if new_class and new_class not in classes:
        classes.append(new_class)
        return f"Added new class: {new_class}"
    elif new_class in classes:
        return "Class already exists."
    else:
        return "No class entered."

def add_new_document_type(new_doc_type):
    new_doc_type = new_doc_type.strip()
    if new_doc_type and new_doc_type not in document_types:
        document_types.append(new_doc_type)
        return f"Added new document type: {new_doc_type}"
    elif new_doc_type in document_types:
        return "Document type already exists."
    else:
        return "No document type entered."

## User Interface with `ipywidgets`
Create interactive widgets for selecting operations, adding new classifications, and controlling processing modes.

In [ ]:
## Updated User Interface with Column Selection Dropdowns

# Define checkboxes for primary operations
translate_german_checkbox = widgets.Checkbox(
    value=False,
    description='Translate to German',
    disabled=False
)

translate_english_checkbox = widgets.Checkbox(
    value=False,
    description='Translate to English',
    disabled=False
)

item_class_checkbox = widgets.Checkbox(
    value=False,
    description='Classify Item',
    disabled=False
)

document_type_checkbox = widgets.Checkbox(
    value=False,
    description='Classify Document Type',
    disabled=False
)

# Define checkboxes for additional classifications
format_checkbox = widgets.Checkbox(
    value=False,
    description='Determine Format',
    disabled=False
)

language_checkbox_widget = widgets.Checkbox(
    value=False,
    description='Identify Language',
    disabled=False
)

creation_place_checkbox = widgets.Checkbox(
    value=False,
    description='Creation Place',
    disabled=False
)

creation_date_checkbox = widgets.Checkbox(
    value=False,
    description='Creation Date',
    disabled=False
)

publication_date_checkbox = widgets.Checkbox(
    value=False,
    description='Publication Date',
    disabled=False
)

number_of_pages_checkbox = widgets.Checkbox(
    value=False,
    description='Number of Pages',
    disabled=False
)

documentary_form_type_checkbox = widgets.Checkbox(
    value=False,
    description='Documentary Form Type',
    disabled=False
)

production_technique_type_checkbox = widgets.Checkbox(
    value=False,
    description='Production Technique Type',
    disabled=False
)

main_subject_checkbox = widgets.Checkbox(
    value=False,
    description='Main Subject',
    disabled=False
)

record_state_checkbox = widgets.Checkbox(
    value=False,
    description='Record State',
    disabled=False
)

title_checkbox = widgets.Checkbox(
    value=False,
    description='Title',
    disabled=False
)

# Define a checkbox for Test Mode
test_mode_checkbox = widgets.Checkbox(
    value=False,
    description='Test Mode (Process 10 Rows)',
    disabled=False
)

# Text inputs for adding new classes
new_class_input = widgets.Text(
    value='',
    placeholder='Enter new class',
    description='New Class:',
    disabled=False
)

add_class_button = widgets.Button(
    description='Add Class',
    disabled=False,
    button_style='',
    tooltip='Click to add new class',
    icon='plus'
)

# Text inputs for adding new document types
new_document_type_input = widgets.Text(
    value='',
    placeholder='Enter new document type',
    description='New Document Type:',
    disabled=False
)

add_document_type_button = widgets.Button(
    description='Add Document Type',
    disabled=False,
    button_style='',
    tooltip='Click to add new document type',
    icon='plus'
)

# Output areas for feedback
class_output = widgets.Output()
document_type_output = widgets.Output()




## Button Click Handlers
Define functions to handle button clicks for adding new classes and document types.

In [ ]:
# Function to handle adding new class
def on_add_class_button_clicked(b):
    with class_output:
        clear_output()
        result = add_new_class(new_class_input.value)
        print(result)

add_class_button.on_click(on_add_class_button_clicked)

# Function to handle adding new document type
def on_add_document_type_button_clicked(b):
    with document_type_output:
        clear_output()
        result = add_new_document_type(new_document_type_input.value)
        print(result)

add_document_type_button.on_click(on_add_document_type_button_clicked)

## Display the Widgets
Organize and display all interactive widgets in a visually appealing layout.

In [ ]:
# Display the dropdowns for column selection
column_selection_ui = widgets.VBox([
    widgets.HTML("<h2>Select Column Mappings:</h2>"),
    widgets.HBox([item_id_dropdown, item_description_dropdown])
])

# Display the widgets including Test Mode
ui = widgets.VBox([
    widgets.HTML("<h2>Select Operations:</h2>"),
    widgets.HBox([translate_german_checkbox, translate_english_checkbox]),
    widgets.HBox([item_class_checkbox, document_type_checkbox]),
    widgets.HBox([format_checkbox, language_checkbox_widget]),
    widgets.HBox([creation_place_checkbox, creation_date_checkbox]),
    widgets.HBox([publication_date_checkbox, number_of_pages_checkbox]),
    widgets.HBox([documentary_form_type_checkbox, production_technique_type_checkbox]),
    widgets.HBox([main_subject_checkbox, record_state_checkbox]),
    widgets.HBox([title_checkbox]),
    column_selection_ui,  # Added Column Selection Dropdowns
    widgets.HTML("<h2>Add New Classifications:</h2>"),
    widgets.HBox([new_class_input, add_class_button]),
    class_output,
    widgets.HBox([new_document_type_input, add_document_type_button]),
    document_type_output,
    widgets.HBox([test_mode_checkbox])  # Added Test Mode Checkbox
])

display(ui)

## Data Processing Function
Handles loading the data, processing it in batches with checkpointing, interacting with the OpenAI API, and saving the results. It also provides detailed feedback on the processing status.

In [ ]:
def process_dataframe(file_path, output_file_path, operations, classes, document_types, formats, languages, production_technique_types, record_states, item_id_column, item_description_column, test_mode=False, batch_size=10, pause_seconds=5):
    # Load the Excel file containing the Hebrew descriptions
    try:
        df = pd.read_excel(file_path)
        logging.info(f"Loaded input file with {df.shape[0]} rows and columns: {df.columns.tolist()}")
    except FileNotFoundError:
        logging.error(f"Input file not found at path: {file_path}")
        return
    except Exception as e:
        logging.error(f"Error loading input file: {e}")
        return

    # Ensure the selected columns exist
    if item_id_column not in df.columns:
        logging.error(f"Selected Item ID column '{item_id_column}' does not exist in the DataFrame.")
        return
    if item_description_column not in df.columns:
        logging.error(f"Selected Item Description column '{item_description_column}' does not exist in the DataFrame.")
        return

    # Convert 'item_id' to string and strip whitespace
    df[item_id_column] = df[item_id_column].astype(str).str.strip()

    # If output file exists, load it
    try:
        df_existing = pd.read_excel(output_file_path)
        df_existing[item_id_column] = df_existing[item_id_column].astype(str).str.strip()
        logging.info(f"Loaded existing output file with {df_existing.shape[0]} rows.")
    except FileNotFoundError:
        df_existing = pd.DataFrame()
        logging.info("No existing output file found. Starting fresh processing.")
    except Exception as e:
        logging.error(f"Error loading existing output file: {e}")
        df_existing = pd.DataFrame()

    # Merge input and existing output DataFrames
    if not df_existing.empty:
        df_merged = df.merge(df_existing, on=item_id_column, how='left', suffixes=('', '_processed'))
    else:
        df_merged = df.copy()

    # Identify rows that need processing
    # Assume 'title' is a field that should be filled after processing
    # Modify 'title' to any key that you expect to be filled after processing
    if 'title' in df_existing.columns:
        rows_to_process = df_merged[df_merged['title'].isna() | df_merged['title'].eq('')]
    else:
        rows_to_process = df_merged.copy()

    # If Test Mode is enabled, limit to the first 10 rows
    if test_mode:
        rows_to_process = rows_to_process.head(10)
        logging.info("Test Mode Enabled: Processing only the first 10 rows.")

    total = rows_to_process.shape[0]
    successes = 0
    skipped = 0

    logging.info(f"Total items to process: {total}")

    # Build the keys list based on selected operations
    keys = []
    if operations['translate_german']:
        keys.append("german_translation")
    if operations['translate_english']:
        keys.append("english_translation")
    if operations['format']:
        keys.append("format")
    if operations['language']:
        keys.append("language")
    if operations['creation_place']:
        keys.append("creation_place")
    if operations['creation_date']:
        keys.append("creation_date")
    if operations['publication_date']:
        keys.append("publication_date")
    if operations['number_of_pages']:
        keys.append("number_of_pages")
    if operations['documentary_form_type']:
        keys.append("documentary_form_type")
    if operations['production_technique_type']:
        keys.append("production_technique_type")
    if operations['main_subject']:
        keys.append("main_subject")
    if operations['record_state']:
        keys.append("record_state")
    if operations['title']:
        keys.append("title")
    if operations['item_class']:
        keys.append("item_class")
    if operations['document_type']:
        keys.append("document_type")

    # Ensure keys are unique
    keys = list(set(keys))

    # Define selected_columns based on original columns and keys
    original_columns = [item_id_column, item_description_column]
    selected_columns = original_columns + keys

    # Prepare a list to collect new processed data
    parsed_rows = []

    # Process in batches to handle large datasets and prevent runtime disconnections
    for start in tqdm(range(0, total, batch_size), desc="Processing Batches"):
        end = min(start + batch_size, total)
        batch = rows_to_process.iloc[start:end]
        logging.debug(f"Processing batch from index {start} to {end}")

        for idx, row in batch.iterrows():
            item_id = str(row[item_id_column]).strip()
            description = row[item_description_column]

            if pd.notnull(description) and str(description).strip() != '':
                extracted, _ = extract_and_translate_skip_on_failure(
                    description,
                    operations,
                    classes,
                    document_types,
                    formats,
                    languages,
                    production_technique_types,
                    record_states
                )
                if extracted:
                    parsed = safe_eval(extracted, keys)
                    # Create a new row dictionary including original and new data
                    new_row = {item_id_column: item_id, item_description_column: description}
                    new_row.update(parsed)
                    parsed_rows.append(new_row)  # Accumulate parsed rows
                    successes += 1
                    logging.debug(f"Successfully processed item ID {item_id}")
                else:
                    skipped += 1
                    logging.debug(f"Skipped item ID {item_id} due to extraction failure.")
            else:
                skipped += 1
                logging.debug(f"No valid description for item ID {item_id}. Skipping.")

        # After processing each batch, append the new data to the existing output DataFrame and save
        if parsed_rows:
            logging.info(f"Number of parsed rows in current batch: {len(parsed_rows)}")
            df_new = pd.DataFrame(parsed_rows, columns=selected_columns)
            if not df_existing.empty:
                df_existing = pd.concat([df_existing, df_new], ignore_index=True)
            else:
                df_existing = df_new.copy()
            # Remove duplicates based on 'item_id'
            df_existing.drop_duplicates(subset=[item_id_column], keep='last', inplace=True)
            try:
                df_existing.to_excel(output_file_path, index=False)
                logging.info(f"Batch {start // batch_size + 1} processed and saved to {output_file_path}.")
                # Create a backup
                backup_path = output_file_path.replace('.xlsx', '_backup.xlsx')
                shutil.copyfile(output_file_path, backup_path)
                logging.info(f"Backup created at {backup_path}")
            except Exception as e:
                logging.error(f"Failed to save output file: {e}")
            parsed_rows = []  # Reset parsed_rows for the next batch
        else:
            logging.info("No new rows parsed in this batch.")

        # Pause between batches to prevent runtime disconnections
        time.sleep(pause_seconds)

    # Display processing summary
    logging.info(f"\nProcessing Summary:")
    logging.info(f"Total Items to Process: {total}")
    logging.info(f"Successfully Processed: {successes}")
    logging.info(f"Skipped Entries: {skipped}")
    logging.info(f"Processed file saved to: {output_file_path}")


## Process Button and Handler
Creates a button that, when clicked, initiates the data processing workflow based on the selected operations and configurations. It also handles resumable processing by skipping already processed entries.

In [ ]:
## Updated Process Button and Handler
## Creates a button that, when clicked, initiates the data processing workflow based on the selected operations and configurations. It also handles resumable processing by skipping already processed entries.

# Create a Button to Trigger the Processing
process_button = widgets.Button(
    description='Process Data',
    disabled=False,
    button_style='success',
    tooltip='Click to process the data with selected operations',
    icon='cogs'
)

output = widgets.Output()

def on_process_button_clicked(b):
    with output:
        clear_output()
        # Define the operations based on current UI selections
        operations = {
            'translate_german': translate_german_checkbox.value,
            'translate_english': translate_english_checkbox.value,
            'item_class': item_class_checkbox.value,
            'document_type': document_type_checkbox.value,
            'format': format_checkbox.value,
            'language': language_checkbox_widget.value,
            'creation_place': creation_place_checkbox.value,
            'creation_date': creation_date_checkbox.value,
            'publication_date': publication_date_checkbox.value,
            'number_of_pages': number_of_pages_checkbox.value,
            'documentary_form_type': documentary_form_type_checkbox.value,
            'production_technique_type': production_technique_type_checkbox.value,
            'main_subject': main_subject_checkbox.value,
            'record_state': record_state_checkbox.value,
            'title': title_checkbox.value
        }

        # Update the classes and document_types with any new additions
        current_classes = classes.copy()
        current_document_types = document_types.copy()

        # Define file paths (ensure these paths are correct)
        input_file_path = '/content/drive/MyDrive/JeckeItems/parsed-items-cleaned.xlsx'  # Update with your file path
        output_file_path = '/content/drive/MyDrive/JeckeItems/Classified_parsed-items-cleaned.xlsx.xlsx'  # Update as needed

        # Check if Test Mode is enabled
        test_mode = test_mode_checkbox.value

        # Retrieve selected columns from dropdowns
        item_id_column = item_id_dropdown.value
        item_description_column = item_description_dropdown.value

        # Validate that the user has selected different columns
        if item_id_column == item_description_column:
            print("Error: 'Item ID Column' and 'Item Description Column' must be different.")
            return

        # Display starting message
        print("Starting data processing...")
        print(f"Selected 'Item ID Column': {item_id_column}")
        print(f"Selected 'Item Description Column': {item_description_column}")

        process_dataframe(
            file_path=input_file_path,
            output_file_path=output_file_path,
            operations=operations,
            classes=current_classes,
            document_types=current_document_types,
            formats=formats,
            languages=languages,
            production_technique_types=production_technique_types,
            record_states=record_states,
            item_id_column=item_id_column,
            item_description_column=item_description_column,
            test_mode=test_mode  # Pass the test_mode flag
        )

process_button.on_click(on_process_button_clicked)

# Display the process button and output area
display(widgets.HBox([process_button]), output)


## Usage Instructions

1. **Select Operations:** Check the boxes corresponding to the tasks you want to perform (e.g., translation, classification).

2. **Add New Classifications (Optional):** Use the text inputs and buttons to add new classes or document types as needed.

3. **Test Mode (Optional):** Enable Test Mode to process only the first 10 rows for testing purposes.

4. **Process Data:** Click the **Process Data** button to start processing. The notebook will process data in batches, saving progress after each batch. If a runtime disconnection occurs, simply re-run the **Process Data** cell to resume processing from the last saved point.

5. **Review Output:** The processed data will be saved to the specified output file path in your Google Drive. Upon completion, a summary of the processing status will be displayed.